## Import libraries

In [1]:
import gc
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Bidirectional, LSTM
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Concatenate, Add, GRU

np.random.seed(42)
tf.random.set_seed(42)

2021-10-26 04:49:20.102375: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


## Load source datasets

In [2]:
train_df = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
print(f"train_df: {train_df.shape}")
train_df.head()

train_df: (6036000, 8)


,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987


In [3]:
test_df = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
print(f"test_df: {test_df.shape}")
test_df.head()

test_df: (4024000, 7)


,id,breath_id,R,C,time_step,u_in,u_out
0,1,0,5,20,0.000000,0.000000,0
1,2,0,5,20,0.031904,7.515046,0
2,3,0,5,20,0.063827,14.651675,0
3,4,0,5,20,0.095751,21.230610,0
4,5,0,5,20,0.127644,26.320956,0


## Feature Engineering

In [4]:
def add_features(df):
    df['cross']= df['u_in'] * df['u_out']
    df['cross2']= df['time_step'] * df['u_out']
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['time_step_cumsum'] = df.groupby(['breath_id'])['time_step'].cumsum()
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    print("Step-1...Completed")
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    print("Step-2...Completed")
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_in__mean'] = df.groupby(['breath_id'])['u_in'].transform('mean')
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    print("Step-3...Completed")
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    print("Step-4...Completed")
    
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] /df['count']
    
    df['breath_id_lag']=df['breath_id'].shift(1).fillna(0)
    df['breath_id_lag2']=df['breath_id'].shift(2).fillna(0)
    df['breath_id_lagsame']=np.select([df['breath_id_lag']==df['breath_id']],[1],0)
    df['breath_id_lag2same']=np.select([df['breath_id_lag2']==df['breath_id']],[1],0)
    df['breath_id__u_in_lag'] = df['u_in'].shift(1).fillna(0)
    df['breath_id__u_in_lag'] = df['breath_id__u_in_lag'] * df['breath_id_lagsame']
    df['breath_id__u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    df['breath_id__u_in_lag2'] = df['breath_id__u_in_lag2'] * df['breath_id_lag2same']
    print("Step-5...Completed")
    
    df['time_step_diff'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
    df['ewm_u_in_mean'] = (df\
                           .groupby('breath_id')['u_in']\
                           .ewm(halflife=9)\
                           .mean()\
                           .reset_index(level=0,drop=True))
    df[["15_in_sum","15_in_min","15_in_max","15_in_mean"]] = (df\
                                                              .groupby('breath_id')['u_in']\
                                                              .rolling(window=15,min_periods=1)\
                                                              .agg({"15_in_sum":"sum",
                                                                    "15_in_min":"min",
                                                                    "15_in_max":"max",
                                                                    "15_in_mean":"mean"})\
                                                               .reset_index(level=0,drop=True))
    print("Step-6...Completed")
    
    df['u_in_lagback_diff1'] = df['u_in'] - df['u_in_lag_back1']
    df['u_out_lagback_diff1'] = df['u_out'] - df['u_out_lag_back1']
    df['u_in_lagback_diff2'] = df['u_in'] - df['u_in_lag_back2']
    df['u_out_lagback_diff2'] = df['u_out'] - df['u_out_lag_back2']
    print("Step-7...Completed")
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    print("Step-8...Completed")
    
    return df


print("Train data...\n")
train = add_features(train_df)

print("\nTest data...\n")
test = add_features(test_df)

del train_df
del test_df
gc.collect()

Train data...

Step-1...Completed
Step-2...Completed
Step-3...Completed
Step-4...Completed
Step-5...Completed
Step-6...Completed
Step-7...Completed
Step-8...Completed

Test data...

Step-1...Completed
Step-2...Completed
Step-3...Completed
Step-4...Completed
Step-5...Completed
Step-6...Completed
Step-7...Completed
Step-8...Completed


0

In [5]:
pressure = train[['pressure']].values.astype('float32')

train.drop(['pressure','id', 'breath_id','one','count',
            'breath_id_lag','breath_id_lag2','breath_id_lagsame',
            'breath_id_lag2same'], axis=1, inplace=True)

test = test.drop(['id', 'breath_id','one','count','breath_id_lag',
                  'breath_id_lag2','breath_id_lagsame',
                  'breath_id_lag2same'], axis=1)

print(f"train: {train.shape} \ntest: {test.shape}")

train: (6036000, 64) 
test: (4024000, 64)


In [6]:
P_MIN = np.min(pressure)
P_MAX = np.max(pressure)
P_STEP = (pressure[1] - pressure[0])[0]
print('Min pressure: {}'.format(P_MIN))
print('Max pressure: {}'.format(P_MAX))
print('Pressure step: {}'.format(P_STEP))
print('Unique values:  {}'.format(np.unique(pressure).shape[0]))

Min pressure: -1.8957443237304688
Max pressure: 64.82099151611328
Pressure step: 0.07030248641967773
Unique values:  950


In [7]:
scaler = RobustScaler().fit(train)
test = scaler.transform(test)

test = test.reshape(-1, 80, train.shape[-1]).astype('float32')
print(f"test: {test.shape}")

del train
gc.collect()

test: (50300, 80, 64)


63

## Keras Model

In [8]:
def head(x_input1, x_input2):
    
    x = Concatenate()([x_input1, x_input2])
    x = Dense(units=256, activation='selu')(x)
    x = Dense(units=128, activation='selu')(x)
    x = Dense(units=64, activation='selu')(x)
    x = Dense(units=1)(x)
    
    return x

In [9]:
base_model1 = load_model("../input/gb-vpp-pulp-fiction/Bidirect_LSTM_model_2021_1C.h5")
base_model1.trainable = False
for layer in base_model1.layers:
    layer._name = 'm1_'+layer.name
    
base_model2 = load_model("../input/gb-vpp-booo-oorreed/Bidirect_LSTM_model_2021_1C.h5")
base_model2.trainable = False
for layer in base_model2.layers:
    layer._name = 'm2_'+layer.name
    
feat_model1 = Model(inputs=base_model1.input, 
                    outputs=base_model1.get_layer(base_model1.layers[-2].name).output,
                    name='gb-vpp-pulp-fiction')

feat_model2 = Model(inputs=base_model2.input, 
                    outputs=base_model2.get_layer(base_model2.layers[-2].name).output,
                    name='gb-vpp-booo-oorreed')

2021-10-26 04:53:16.345943: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-26 04:53:16.352051: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-26 04:53:16.399618: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-26 04:53:16.400331: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-10-26 04:53:16.400396: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-26 04:53:16.429413: I tensorflow/stream_executor/platform/def

In [10]:
x_input = Input(shape=(test.shape[-2:]))
rd_model = Model(x_input, head(feat_model1(x_input), feat_model2(x_input)),
                 name='DNN_Model')
rd_model.summary()

Model: "DNN_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 64)]     0                                            
__________________________________________________________________________________________________
gb-vpp-pulp-fiction (Functional (None, 80, 128)      26288000    input_1[0][0]                    
__________________________________________________________________________________________________
gb-vpp-booo-oorreed (Functional (None, 80, 192)      27671232    input_1[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 80, 320)      0           gb-vpp-pulp-fiction[0][0]        
                                                                 gb-vpp-booo-oorreed[0][0]

## Keras DNN Model Predictions

In [11]:
BATCH_SIZE = 512
test_preds = []

for fold in range(7):
    print(f"\n>>>>>  Fold-{fold+1}  <<<<<")
    
    rd_model.load_weights(f'../input/gb-vpp-reservoir-dogs/Bidirect_LSTM_model_{fold+1}C.h5')
    test_preds.append(rd_model.predict(test, batch_size=BATCH_SIZE, verbose=1).squeeze().reshape(-1, 1).squeeze())
    
    model = load_model(f'../input/gb-vpp-show-me-the-model/Bidirect_LSTM_model_2021_{fold+1}C.h5')
    test_preds.append(model.predict(test, batch_size=BATCH_SIZE, verbose=1).squeeze().reshape(-1, 1).squeeze())
    
    model = load_model(f'../input/gb-vpp-booo-oorreed/Bidirect_LSTM_model_2021_{fold+1}C.h5')
    test_preds.append(model.predict(test, batch_size=BATCH_SIZE, verbose=1).squeeze().reshape(-1, 1).squeeze())
    
    #model = load_model(f'../input/gb-vpp-long-live-and-prosper/Bidirect_LSTM_model_2021_{fold+1}C.h5')
    #test_preds.append(model.predict(test, batch_size=BATCH_SIZE, verbose=1).squeeze().reshape(-1, 1).squeeze())
    
    model = load_model(f'../input/gb-vpp-pulp-fiction/Bidirect_LSTM_model_2021_{fold+1}C.h5')
    test_preds.append(model.predict(test, batch_size=BATCH_SIZE, verbose=1).squeeze().reshape(-1, 1).squeeze())
    
    model = load_model(f'../input/gb-vpp-new-dawn/Bidirect_LSTM_model_{fold+1}C.h5')
    test_preds.append(model.predict(test, batch_size=BATCH_SIZE, verbose=1).squeeze().reshape(-1, 1).squeeze())


>>>>>  Fold-1  <<<<<


2021-10-26 04:53:50.719293: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1030144000 exceeds 10% of free system memory.
2021-10-26 04:53:51.516627: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-26 04:53:51.527521: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000170000 Hz
2021-10-26 04:54:01.079005: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-10-26 04:54:01.881491: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-10-26 04:54:01.924068: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


99/99 [==============================] - 97s 823ms/step


2021-10-26 04:55:37.423888: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1030144000 exceeds 10% of free system memory.


99/99 [==============================] - 33s 298ms/step


...

99/99 [==============================] - 47s 422ms/step


2021-10-26 04:57:19.624653: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1030144000 exceeds 10% of free system memory.


99/99 [==============================] - 45s 402ms/step


2021-10-26 04:58:15.694514: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1030144000 exceeds 10% of free system memory.


99/99 [==============================] - 47s 428ms/step

>>>>>  Fold-2  <<<<<
99/99 [==============================] - 47s 428ms/step

>>>>>  Fold-3  <<<<<
99/99 [==============================] - 47s 428ms/step

>>>>>  Fold-4  <<<<<
99/99 [==============================] - 47s 428ms/step

>>>>>  Fold-5  <<<<<
99/99 [==============================] - 47s 427ms/step

>>>>>  Fold-6  <<<<<
99/99 [==============================] - 47s 429ms/step

>>>>>  Fold-7  <<<<<
99/99 [==============================] - 46s 427ms/step


## Create submission file

In [12]:
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')
submission["pressure"] = np.median(np.vstack(test_preds),axis=0)
submission["pressure"] = np.round((submission.pressure - P_MIN)/P_STEP) * P_STEP + P_MIN
submission["pressure"] = np.clip(submission.pressure, P_MIN, P_MAX)
submission.to_csv('median_submission.csv', index=False)